# Finding the Right Dose

David Augustin

In [1]:
#
# Slide 1: Scatter plot of control tumour growth.
#

import os

import pandas as pd
import plotly.graph_objects as go


# Create figure
fig = go.Figure()

# Import data
# Get path of directory one up current working directory
path = os.path.dirname(os.getcwd())

# Import LXF A677 control growth data
lxf_data = pd.read_csv(path + '/notebooks/control_growth/data/lxf_control_growth.csv')

# Scatter plot LXF A677 time-series data for each mouse
mouse_ids = lxf_data['#ID'].unique()
for id_m in mouse_ids:
    # Create mask for mouse
    mask = lxf_data['#ID'] == id_m

    # Get time points for mouse
    times = lxf_data['TIME in day'][mask]

    # Get observed tumour volumes for mouse
    observed_volumes = lxf_data['TUMOUR VOLUME in cm^3'][mask]

    # Plot data
    fig.add_trace(go.Scatter(
        x=times,
        y=observed_volumes,
        name="ID: %s" % id_m,
        showlegend=True,
        hovertemplate=
            "<b>ID: %d</b><br>" % id_m +
            "Cancer type: LXF A677<br>"
            "Time: %{x:} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "<extra></extra>",
        mode="markers",
        marker=dict(
            symbol='circle',
            opacity=0.7,
            line=dict(color='black', width=1))
    ))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    xaxis_title='Time in day',
    yaxis_title='Tumour volume in cm^3',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Show figure
fig.show()

Notes:
- Describe axes
- Colours indicate individuals
- The trend is the same
- BUT there are subsantial differences bewtween the individuals

### There is substantial variation between individuals

In [17]:
#
# Slide 3: Scatter plot of treated mice.
#

import os

import numpy as np
import pandas as pd
import plotly.colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pkpd.utils


path = os.path.dirname(os.getcwd())

# Import LXF A677 Erlotinib low dose data
low_dose_data = pd.read_csv(
    path + '/notebooks/lung_cancer/erlotinib/treatment_low_dose/data/erlotinib_low_dose_lxf.csv')

# Import LXF A677 Erlotinib intermediate dose data
inter_dose_data = pd.read_csv(
    path + '/notebooks/lung_cancer/erlotinib/treatment_intermediate_dose/data/erlotinib_intermediate_dose_lxf.csv')

# Get number of individual mice
n_mice = len(low_dose_data['#ID'].unique())

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Create figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.3, 0.7], vertical_spacing=0.05)

# Scatter plot of concentration and tumour growth data
for index, mouse_id in enumerate(np.sort(low_dose_data['#ID'].unique())):
    # Mask dataset for mouse
    mask = low_dose_data['#ID'] == mouse_id
    mouse_data = low_dose_data[mask]

    # Get tumour volume measurement times
    volume_times = mouse_data['TIME VOLUME in day'].to_numpy()

    # Get measured concentrations
    volumes = mouse_data['TUMOUR VOLUME in cm^3'].to_numpy()

    # Get dosing time points
    dose_times = mouse_data['TIME DOSE in day'].to_numpy()

    # Get doses
    doses = mouse_data['DOSE AMOUNT in mg'].to_numpy()

    # Filter nans from dose arrays
    dose_times = dose_times[~np.isnan(dose_times)]
    doses = doses[~np.isnan(doses)]

    # Convert dose events to cumulative dose amount time series
    dose_times, doses = pkpd.utils.compute_cumulative_dose_amount(
        times=dose_times,
        doses=doses,
        end_exp=30)

    # Plot cumulative dosed amount
    fig.add_trace(
        go.Scatter(
            x=dose_times,
            y=doses,
            legendgroup="ID: %d" % mouse_id,
            name="ID: %d" % mouse_id,
            showlegend=False,
            hovertemplate=
                "<b>Cumulative dose in mg</b><br>" +
                "ID: %d<br>" % mouse_id +
                "Time: %{x:.0f} day<br>" +
                "Tumour volume: %{y:.02f} cm^3<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(color=colors[index])),
        row=1,
        col=1)

    # Plot tumour volume data
    fig.add_trace(
        go.Scatter(
            x=volume_times,
            y=volumes,
            legendgroup="ID: %d" % mouse_id,
            name="ID: %d" % mouse_id,
            showlegend=True,
            hovertemplate=
                "<b>Tumour volume in cm^3</b><br>" +
                "ID: %d<br>" % mouse_id +
                "Time: %{x:} day<br>" +
                "Tumour volume: %{y:.02f} cm^3<br>" +
                "<extra></extra>",
            mode="markers",
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1),
                color=colors[index])),
        row=2,
        col=1)

# Get number of individual mice
n_mice = len(inter_dose_data['#ID'].unique())

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Scatter plot of concentration and tumour growth data
for index, mouse_id in enumerate(np.sort(inter_dose_data['#ID'].unique())):
    # Mask dataset for mouse
    mask = inter_dose_data['#ID'] == mouse_id
    mouse_data = inter_dose_data[mask]

    # Get tumour volume measurement times
    volume_times = mouse_data['TIME VOLUME in day'].to_numpy()

    # Get measured concentrations
    volumes = mouse_data['TUMOUR VOLUME in cm^3'].to_numpy()

    # Get dosing time points
    dose_times = mouse_data['TIME DOSE in day'].to_numpy()

    # Get doses
    doses = mouse_data['DOSE AMOUNT in mg'].to_numpy()

    # Filter nans from dose arrays
    dose_times = dose_times[~np.isnan(dose_times)]
    doses = doses[~np.isnan(doses)]

    # Convert dose events to cumulative dose amount time series
    dose_times, doses = pkpd.utils.compute_cumulative_dose_amount(
        times=dose_times,
        doses=doses,
        end_exp=30)

    # Plot cumulative dosed amount
    fig.add_trace(
        go.Scatter(
            x=dose_times,
            y=doses,
            legendgroup="ID: %d" % mouse_id,
            name="ID: %d" % mouse_id,
            showlegend=False,
            visible=False,
            hovertemplate=
                "<b>Cumulative dose in mg</b><br>" +
                "ID: %d<br>" % mouse_id +
                "Time: %{x:.0f} day<br>" +
                "Tumour volume: %{y:.02f} cm^3<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(color=colors[index])),
        row=1,
        col=1)

    # Plot tumour volume data
    fig.add_trace(
        go.Scatter(
            x=volume_times,
            y=volumes,
            legendgroup="ID: %d" % mouse_id,
            name="ID: %d" % mouse_id,
            showlegend=True,
            visible=False,
            hovertemplate=
                "<b>Tumour volume in cm^3</b><br>" +
                "ID: %d<br>" % mouse_id +
                "Time: %{x:} day<br>" +
                "Tumour volume: %{y:.02f} cm^3<br>" +
                "<extra></extra>",
            mode="markers",
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1),
                color=colors[index])),
        row=2,
        col=1)

# Set figure size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    template="plotly_white")

# Set X axis label
fig.update_xaxes(title_text='Time in day', row=2, col=1)

# Set Y axes labels
fig.update_yaxes(title_text='Amount in mg', row=1, col=1)
fig.update_yaxes(title_text='Tumour volume in cm^3', row=2, col=1)

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis2.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis2.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        dict(
            type = "buttons",
            direction = "down",
            buttons=list([
                dict(
                    args=[{"visible": [True]*16 + [False]*16}],
                    label="6.25 mg/kg/day",
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*16 + [True]*16}],
                    label="25 mg/kg/day",
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=1.07,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()
